In [1]:
import pickle
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, neighbors, datasets
from sklearn import svm
import scipy.signal as signal
import numpy as np
from sklearn import svm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import LeavePOut
from sklearn.preprocessing import StandardScaler
import sys
import time
import os
import shutil
from sklearn.externals import joblib
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
def split_matrix_two_blocks(y, percentage1, percentage2, seed):
    """Build k indices for k-fold."""
    if(percentage1+percentage2==1):
        num_row = len(y)
        #print(num_row)
        interval_1 = int(percentage1*num_row);
        
        np.random.seed(seed)
        indices = np.random.permutation(num_row);
        first_indices = indices[0:interval_1];
        second_indices = indices[interval_1:num_row];
        return [np.array(first_indices),np.array(second_indices)]
    else:
        print('>>>>>>>>>>>ERROR:Not valid splitting percentage')
        
        
##
## This function reutrn a list of matrices. Each matrix correspond to a question instance in which each row is a channel, and in the coloumn it develop the signal in time
## The function also manage to standardize the time length
def channels_to_vector(channels): 
    time_instances=[];
    dim=channels.shape;
    #find the length min of the signal in the specified temporal instance
    length_min=len(channels[0,1]);
    for i in range (0,dim[1]):
        single_measurement=channels[0,i];
        single_length=single_measurement.shape[0]
        if(single_length<length_min):
                length_min=single_length;
    #export the signals
    for i in range (0,dim[1]):
        single_measurement=channels[0,i];
        dim1=single_measurement.shape;
        time_instance=[];
        for j  in range (0,dim1[1]):
            if(len(single_measurement[:,j])>length_min):
                single_signal=single_measurement[:,j][0:length_min]
            else:
                single_signal=single_measurement[:,j]
            #put in a list 
            time_instance.append(np.asarray(single_signal).reshape(len(single_signal),1).T);
       # create the matrix of the signals per a single time instance 
        time_instance=np.concatenate(time_instance);
        time_instances.append(time_instance);   
    return time_instances;


##
# Create the train data matrix
##
## usage
def get_feature_matrix_and_labels(channel_structure,label,features_extracted,connectivity_feature):
    list_train=[]
    list_labels=[]
    cont=0;
    index_connectivity=0;
    list_row=[]
    
    for time_instance in channel_structure:
        dim1=time_instance.shape
        #indipendent_components=extract_ICs(time_instance,n_ICA_components);
        for j  in range (0,dim1[0]):
           
            features=features_extracted[cont,:];
            list_row.append(features);
            cont=cont+1;
        list_row.append(connectivity_feature[index_connectivity,:]);
        index_connectivity=index_connectivity+1;
        labels=get_labels(1,label);
        feature_row=np.concatenate(list_row);
        list_train.append(feature_row.reshape(len(feature_row),1).T)
        list_labels.append(labels);
        list_row=[]
        
    train_TX=np.concatenate(list_train)
    labels=np.concatenate(list_labels,axis=0)
    
    return train_TX,labels.T.reshape(labels.size)


### Description
def get_labels(number, string):
    if(string=="No"):
        return np.zeros(number)    
    if(string=="Yes"):
        return np.ones(number)
    
## description
def select_features(weights,matrix,th):
    cont=0;
    i=0;
    while(cont<len(weights)):
        if(weights[cont]<th):

            mask = np.ones(matrix.shape[1], dtype=bool)
            mask[i] = False
            matrix=matrix[:,mask]
        else:
            i=i+1;
        cont=cont+1;
    return matrix


def get_accuracy(predicted_labels, true_labels):
     if (predicted_labels.size == true_labels.size):
        return  np.sum(predicted_labels ==  true_labels )/len( true_labels)
    
    

def classification_SVM_experiments_std(X, Y, classifier, fraction_train_test, num_experiments,Model_Name,save_model):
    
    seed=range(num_experiments)
    svm_total_acc_test  = []
    svm_total_acc_train = [] 
    dataset_length=X.shape[0];
    
    for single_seed in seed:
        scaler = StandardScaler()
        
        [i1,i2]=split_matrix_two_blocks(X, fraction_train_test, 1-fraction_train_test,single_seed)

        
        train =X[i1,:]
        labels_train=Y[i1]
        
        test = X[i2,:]
        labels_test=Y[i2]
        
        scaler.fit(train)
        train = scaler.transform(train)
        test = scaler.transform(test)
  
        clf=classifier
        clf.fit(train, labels_train)  

        
        #Accuracy on test
        predicted_labels_test = clf.predict(test)
        SVM_accuracy_test = get_accuracy(predicted_labels_test, labels_test)
        svm_total_acc_test.append(SVM_accuracy_test)
        
        if(save_model==1):
            old_path=os.getcwd()
            os.chdir('Model'+Model_Name)
            joblib.dump(clf, 'Classifier_'+Model_Name+'.pkl') 
            os.chdir(old_path)
    
        #Accuracy on train
        predicted_labels_train = clf.predict(train)
        SVM_accuracy_train = get_accuracy(predicted_labels_train, labels_train)
        svm_total_acc_train.append(SVM_accuracy_train)
        #print("Accuracy: "+ str(SVM_accuracy) + "; iteration  " + str(single_seed) )
    return svm_total_acc_test, svm_total_acc_train

def performance_assesment_fraction_std(X, Y, num_experiment, classifier,Model_Name):
    fracs = np.linspace(0.25,0.9,X.shape[0])
    accuracy_test_mean  = []
    accuracy_test_std   = []
    accuracy_train_mean = []
    accuracy_train_std  = []

    for frac_tr_te in fracs:
        #print("Evaluation progress: " + str(int((frac_tr_te-fracs[0])/(fracs[-1]-fracs[0])*100)) + " %")
        acc_test, acc_train = classification_SVM_experiments_std(X, Y, classifier, frac_tr_te, num_experiment,Model_Name,0)
        #saving of metrics of interest
        accuracy_test_mean.append(np.mean(acc_test))
        accuracy_test_std.append(np.std(acc_test))
        accuracy_train_mean.append(np.mean(acc_train))
        accuracy_train_std.append(np.std(acc_train))
    
    frac_tr_te=0.95;
    acc_test, acc_train =classification_SVM_experiments_std(X, Y, classifier, frac_tr_te, 1,Model_Name,1)
    #plot the figure
    plt.figure(figsize=(10, 7), dpi=80)
    plt.errorbar(np.floor(fracs*60), accuracy_test_mean, yerr=accuracy_test_std, label="Error bars plot", fmt="s-",  linewidth=3)
    plt.errorbar(np.floor(fracs*60), accuracy_train_mean, yerr=accuracy_train_std, label="Error bars plot", fmt="s-",  linewidth=3)
    plt.grid(b=True, which='major', color='k', linestyle='--', alpha = 0.4)
    plt.minorticks_on()
    plt.title('SVM perfomances over different train/test dataset of reduced features')
    plt.ylabel('Accuracy')
    plt.xlabel('Train instances considered')

    plt.legend(['Test Accuracy', 'Train Accuracy'], loc=4)
    name='train_test_acc_fine_tuned_IID_'+Model_Name+'.eps'
    plt.savefig(name, format='eps')
    plt.show()
    
def split_train_test_temporal(X, y, n):
    #n is the number of instances of yes and no, n>1
    half = int(X.shape[0]/2)
    if (n>half): n=half
    X_tr = np.concatenate(( X[0:n,:] , X[half:half+n,:] ))
    X_te = np.concatenate(( X[n:half,:], X[half+n:,:] ))
    y_tr = np.concatenate(( y[0:n]   , y[half:half+n]   ))
    y_te = np.concatenate(( y[n:half]  , y[half+n:]   ))
    
    return X_tr, X_te, y_tr, y_te

def get_experimental_sequence(X, y, order, c):
    idx_tr = []
    y_tr = []
    half = int(X.shape[0]/2)
    #c = number of minimum instances
    i=0
    j=0
    for element in order:
        if have_two_classes(y_tr) and len(y_tr)>c:
            break
        if element==1: # 1 is YES
            idx_tr.append(i)
            #X_tr.append(X[i,:])
            y_tr.append(y[i])
            i+=1
        else: # 0 is NO
            idx_tr.append(j+half)
            #X_tr.append(X[half+j,:])
            y_tr.append(y[half+j])
            j+=1
            
    X_tr = X[idx_tr,:]
    y_tr = y[idx_tr]
    
    idx_te = [k for k in range(X.shape[0])]
    for l in idx_tr:
        idx_te.remove(l)
    X_te = X[idx_te,:]
    y_te = y[idx_te]
    
    return X_tr, X_te, y_tr, y_te

def have_two_classes(y):
    have_zero = False
    have_one = False
    for element in y: 
        if element==1: have_zero = True
    for element in y: 
        if element==0: have_one = True
    return have_zero & have_one



def performance_assesment_instances_std(X, y, order, Model_Name):
    feature_reduced = X
    labels = y
    #fracs = np.linspace(0.2,0.9,60)
    tot_perf = []
    start = 4
    ran = range(start,int(X.shape[0]/2))

    for n in ran:
     
        X_tr, X_te, y_tr, y_te = get_experimental_sequence(feature_reduced, labels, order, n)
        #print(X_te.shape)
        #print(X_te)
        #Scale the data to 0-mean and 1-variance
        scaler = StandardScaler()
        X_tr_std = scaler.fit_transform(X_tr)
        X_te_std = scaler.transform(X_te)
        #print(n)
        #print(X_tr_std.shape)
        #print(X_te_std.shape)
        #Create the model
        final_clf = svm.SVC(C = 1, kernel = 'linear', gamma = 'auto')
        #SVM fit on the training data (reduced and scaled)
        final_clf.fit(X_tr_std, y_tr)
        #performances on test
        tot_perf.append(final_clf.score(X_te_std, y_te))
        

    #plot the figure
    plt.figure(figsize=(10, 7), dpi=80)
    plt.errorbar(range(start, X.shape[0]-4, 2) , tot_perf, label="Error bars plot", fmt="s-",  linewidth=3)
    #plt.errorbar(range(40)+1, accuracy_train_mean, label="Error bars plot", fmt="s-",  linewidth=3)
    plt.grid(b=True, which='major', color='k', linestyle='--', alpha = 0.4)
    plt.minorticks_on()
    plt.title('SVM perfomances over different train/test dataset of reduced features')
    plt.ylabel('Accuracy')
    plt.xlabel('Train instances considered')

    plt.legend(['Test Accuracy', 'Train Accuracy'], loc=4)
    name='test_acc_CORR_'+Model_Name+'.eps'
    plt.savefig(name, format='eps')
    plt.show()
    
    X_tr, X_te, y_tr, y_te = get_experimental_sequence(feature_reduced, labels, order, 19)
    
    scaler = StandardScaler()
    X_tr_std = scaler.fit_transform(X_tr)
    X_te_std = scaler.transform(X_te)

    #Create the model
    final_clf = svm.SVC(C = 1, kernel = 'linear', gamma = 'auto')
    #SVM fit on the training data (reduced and scaled)
    final_clf.fit(X_tr_std, y_tr)
    #performances on test
    one_session_train_score = final_clf.score(X_te_std, y_te)
    print('Score on one session of training data (20 questions) for ' +str(Model_Name) +': ' + str(one_session_train_score))
    
    
def get_experimental_sequence(X, y, order, c):
    idx_tr = []
    y_tr = []
    half = int(X.shape[0]/2)
    #c = number of minimum instances
    i=0
    j=0
    for element in order:
        if have_two_classes(y_tr) and len(y_tr)>c:
            break
        if element==1: # 1 is YES
            idx_tr.append(i)
            #X_tr.append(X[i,:])
            y_tr.append(y[i])
            i+=1
        else: # 0 is NO
            idx_tr.append(j+half)
            #X_tr.append(X[half+j,:])
            y_tr.append(y[half+j])
            j+=1
            
    X_tr = X[idx_tr,:]
    y_tr = y[idx_tr]
    
    idx_te = [k for k in range(X.shape[0])]
    for l in idx_tr:
        idx_te.remove(l)
    X_te = X[idx_te,:]
    y_te = y[idx_te]
    
    return X_tr, X_te, y_tr, y_te

def have_two_classes(y):
    have_zero = False
    have_one = False
    for element in y: 
        if element==1: have_zero = True
    for element in y: 
        if element==0: have_one = True
    return have_zero & have_one


def extract_dataset(destination_folder):
    #Import data from mat files
    old_path=os.getcwd()
    os.chdir(destination_folder)
    yes_EEG_contents = sio.loadmat('EEGyes.mat')
    no_EEG_contents = sio.loadmat('EEGno.mat')

    channels_no_EEG=no_EEG_contents["EEGno"]
    channels_yes_EEG=yes_EEG_contents["EEGyes"]

    #Features Loading
    features_extracted_yes   = sio.loadmat('FeaturesYes.mat')['FeaturesYes']
    features_extracted_no    = sio.loadmat('FeaturesNO.mat')['FeaturesNo']
    connectivity_feature_yes = sio.loadmat('ConnectivityFeaturesYes.mat')['ConnectivityFeaturesYes']
    connectivity_feature_no  = sio.loadmat('ConnectivityFeaturesNo.mat')['ConnectivityFeaturesNo']

    channels_structure_yes_EEG = channels_to_vector(channels_yes_EEG)
    channels_structure_no_EEG  = channels_to_vector(channels_no_EEG)

    ##Structuring of the data:
    #the code below create the train matrix with respect to the signal given in "channel_structure" but using the features contained in "features_extracted*" and in "connettivity_feature*".
    feature_dataset_yes_EEG, EEG_yes_labels = get_feature_matrix_and_labels(channels_structure_yes_EEG,"Yes",features_extracted_yes,connectivity_feature_yes);

    feature_dataset_no_EEG, EEG_no_labels = get_feature_matrix_and_labels(channels_structure_no_EEG,"No",features_extracted_no,connectivity_feature_no);

    #Merge the labeled data
    feature_dataset_full = np.concatenate((feature_dataset_yes_EEG, feature_dataset_no_EEG), axis=0 )
    labels = np.concatenate((EEG_yes_labels,EEG_no_labels), axis=0)
    
    #Order
    order = sio.loadmat('order.mat')['order']
    order = order.flatten()

    os.chdir(old_path)
    
   

    return feature_dataset_full, labels, order

def select_k_iid(X, y):
    feature_dataset_full = X
    labels = y
    tot_perf = []
    end_feat = 100
    for k in range(1,end_feat):
        #print report
        #sys.stdout.write("\033[F") # Cursor up one line
        #sys.stdout.write("\033[K") # Clear to the end of line
        #print('progress: ' + str(k/end_feat*100) + ' %')


        #reduce with anova f-test score the data in k features
        reducer = SelectKBest(f_classif, k)
        feature_reduced = reducer.fit_transform(feature_dataset_full, labels)


        clf = svm.SVC(C = 1, kernel = 'linear', gamma = 'auto') #Parameters of the SVM
        fraction_train_test = 0.30 #Division tran/test of the full dataset
        num_experiments = 500 #Number of different partition for better evaluating the performances

        #Function that actually computes the performances on the k features 
        perf_temp = classification_SVM_experiments_std(feature_reduced, labels, clf, fraction_train_test, num_experiments,Model_Name,0) #Note: this method scale the data to have 0-mean and unit variance
        tot_perf.append(np.mean(perf_temp)) #Save the result


    best_k = np.argmax(tot_perf)+1
    print("smallest k that gives best low train results:")
    print(best_k)
    print()
    print("wich have lead to the top performance:")
    print(np.max(tot_perf))
    return best_k

def select_k_temporal(X, y, order, n):
    feature_dataset_full = X
    labels = y
    tot_perf = []
    end_feat = 100
    for k in range(1,end_feat):




        #X_tr, X_te, y_tr, y_te = split_train_test_temporal(feature_reduced, labels, 10)
        X_tr, X_te, y_tr, y_te = get_experimental_sequence(feature_dataset_full, labels, order, n)
        
        #reduce with anova f-test score the data in k features
        reducer = SelectKBest(f_classif, k)
        X_tr = reducer.fit_transform(X_tr, y_tr)
        X_te = reducer.transform(X_te)
        
        #Scale the data to 0-mean and 1-variance
        scaler = StandardScaler()
        X_tr_std = scaler.fit_transform(X_tr)
        X_te_std = scaler.transform(X_te)

        #Create the model
        final_clf = svm.SVC(C = 1, kernel = 'linear', gamma = 'auto')
        #SVM fit on all the available data (reduced and scaled)
        final_clf.fit(X_tr_std, y_tr)
        #performances
        tot_perf.append(final_clf.score(X_te_std, y_te))



    best_k = np.argmax(tot_perf)+1
    print("smallest k that gives best low train results:")
    print(best_k)
    print()
    print("wich have lead to the top performance:")
    print(np.max(tot_perf))
    return best_k

# Interesting stuff:

# Analysis of results

Here we use 20 instaces for train (one session) and the remaining for the test.

In [3]:
# Load Data

########################### INPUT PARAMETERS########

#Select the traine  classifier


###################################################

#Select the input folders to test the model
Input_Path_Folders=['./DataDay1','./DataDay2','./DataDay3','./DataDay4','./DataDay5','./DataDay6']
i = 1
n=19 # number of questions -1
#print('Model trained: ')
for single_input in Input_Path_Folders:
    day = 'Day_'+str(i)
    print('Accuracy on dataset gathered on: ' + day)
    [feature_dataset_full, labels, order]= extract_dataset(single_input)
    best_k_temporal = select_k_temporal(feature_dataset_full, labels, order, n) #Best K iid
    
    #Reduce dimensions
    reducer = SelectKBest(f_classif, best_k_temporal)
    feature_dataset_reduced = reducer.fit_transform(feature_dataset_full, labels)
    
    X_tr, X_te, y_tr, y_te = get_experimental_sequence(feature_dataset_reduced, labels, order, n)
    print('Training made on ' + str(X_tr.shape[0])+ ' instances')
    print('Test made on ' + str(X_te.shape[0])+ ' instances')
    scaler = StandardScaler()
    X_tr_std = scaler.fit_transform(X_tr)
    X_te_std = scaler.transform(X_te)

    #Create the model
    final_clf = svm.SVC(C = 1, kernel = 'linear', gamma = 'auto')
    #SVM fit on the training data (reduced and scaled)
    final_clf.fit(X_tr_std, y_tr)
    #performances on test
    one_session_train_score = final_clf.score(X_te_std, y_te)
    print('Score for ' +str(day) +': ' + str(one_session_train_score))
    print()
    print()
    
    i+=1
    
    

Accuracy on dataset gathered on: Day_1
smallest k that gives best low train results:
36

wich have lead to the top performance:
0.666666666667
Training made on 20 instances
Test made on 60 instances
Score for Day_1: 0.616666666667


Accuracy on dataset gathered on: Day_2
smallest k that gives best low train results:
5

wich have lead to the top performance:
0.666666666667
Training made on 20 instances
Test made on 60 instances
Score for Day_2: 0.616666666667


Accuracy on dataset gathered on: Day_3
smallest k that gives best low train results:
2

wich have lead to the top performance:
0.7
Training made on 20 instances
Test made on 20 instances
Score for Day_3: 0.7


Accuracy on dataset gathered on: Day_4
smallest k that gives best low train results:
67

wich have lead to the top performance:
0.55
Training made on 20 instances
Test made on 60 instances
Score for Day_4: 0.6


Accuracy on dataset gathered on: Day_5
smallest k that gives best low train results:
25

wich have lead to the to

# Analysis of the performances with 2 session of training

Note: day 3 was excluded because it has only 2 session.

In [4]:
# Load Data

########################### INPUT PARAMETERS########

#Select the traine  classifier


###################################################

#Select the input folders to test the model
Input_Path_Folders=['./DataDay1','./DataDay2','./DataDay4','./DataDay5','./DataDay6']

n = 39
#print('Model trained: ')
for single_input in Input_Path_Folders:
    #day = 'Day_'+str(i)
    day = single_input
    print('Accuracy on dataset gathered on: ' + day)
    [feature_dataset_full, labels, order]= extract_dataset(single_input)
    best_k_temporal = select_k_temporal(feature_dataset_full, labels, order, n) #Best K iid
    
    #Reduce dimensions
    reducer = SelectKBest(f_classif, best_k_temporal)
    feature_dataset_reduced = reducer.fit_transform(feature_dataset_full, labels)
    
    X_tr, X_te, y_tr, y_te = get_experimental_sequence(feature_dataset_reduced, labels, order, n)
    print('Training made on ' + str(X_tr.shape[0])+ ' instances')
    print('Test made on ' + str(X_te.shape[0])+ ' instances')
    scaler = StandardScaler()
    X_tr_std = scaler.fit_transform(X_tr)
    X_te_std = scaler.transform(X_te)

    #Create the model
    final_clf = svm.SVC(C = 1, kernel = 'linear', gamma = 'auto')
    #SVM fit on the training data (reduced and scaled)
    final_clf.fit(X_tr_std, y_tr)
    #performances on test
    one_session_train_score = final_clf.score(X_te_std, y_te)
    print('Score for ' +str(day) +': ' + str(one_session_train_score))
    print()
    print()
    
    i+=1
    

Accuracy on dataset gathered on: ./DataDay1
smallest k that gives best low train results:
6

wich have lead to the top performance:
0.7
Training made on 40 instances
Test made on 40 instances
Score for ./DataDay1: 0.75


Accuracy on dataset gathered on: ./DataDay2
smallest k that gives best low train results:
22

wich have lead to the top performance:
0.65
Training made on 40 instances
Test made on 40 instances
Score for ./DataDay2: 0.55


Accuracy on dataset gathered on: ./DataDay4
smallest k that gives best low train results:
28

wich have lead to the top performance:
0.775
Training made on 40 instances
Test made on 40 instances
Score for ./DataDay4: 0.9


Accuracy on dataset gathered on: ./DataDay5
smallest k that gives best low train results:
23

wich have lead to the top performance:
1.0
Training made on 40 instances
Test made on 20 instances
Score for ./DataDay5: 1.0


Accuracy on dataset gathered on: ./DataDay6
smallest k that gives best low train results:
45

wich have lead to 